In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
from operator import add

In [4]:
conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate()

In [5]:
def rdd_to_dictionary(rdd):
    return sc.parallelize(rdd).collectAsMap()

In [6]:
def write_json(dictionary,filename):
    import json  

    with open(filename, "w") as outfile:  
        json.dump(dictionary, outfile)    

In [7]:
documents = ["data/4300-0.txt","data/pg19033.txt", "data/pg36.txt","data/pg514.txt","data/pg1497.txt","data/pg3207.txt","data/pg42671.txt","data/pg6130.txt"]
data_bag = spark.read.text(documents).rdd.map(lambda r: r[0])

In [8]:
words_count = data_bag.flatMap(lambda x: x.strip().split())\
                .map(lambda x: x.lower()) \
                .map(lambda x: (x, 1))\
                .reduceByKey(add)\
                .filter(lambda x: x[1] > 1)
# type(words_count.collect())

In [9]:
top_40_words = words_count.takeOrdered(40, key = lambda x: -x[1])
# type(top_40_words)

In [10]:
top_40_words_dictionary = rdd_to_dictionary(top_40_words)
print(top_40_words_dictionary)


{'the': 78844, 'and': 45168, 'of': 44739, 'to': 33436, 'a': 24234, 'in': 22126, 'that': 14818, 'he': 13019, 'is': 12918, 'his': 12270, 'i': 11044, 'with': 10296, 'for': 10036, 'as': 9639, 'be': 8834, 'was': 8787, 'not': 8141, 'it': 8123, 'but': 7856, 'by': 7701, 'or': 7407, 'her': 7403, 'they': 6735, 'which': 6517, 'you': 6354, 'on': 6214, 'from': 5811, 'at': 5695, 'are': 5590, 'she': 5458, 'all': 5437, 'their': 5285, 'have': 5146, 'had': 4647, 'this': 4090, 'my': 3841, 'so': 3710, 'we': 3629, 'no': 3620, 'if': 3571}


In [11]:
write_json(top_40_words_dictionary,"sp1.json")

In [12]:
sc.stop()